In [1]:
! pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Datasets de peliculas y datos de reseñas y etiquetas

In [6]:
import requests, zipfile, io, os
import tarfile
import pandas as pd
import time

In [3]:
url = "https://files.grouplens.org/datasets/movielens/ml-32m.zip"
response = requests.get(url)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("movies-data")

In [10]:
informacion = ["links", "movies", "ratings", "tags"]
df_list = []

for i in informacion:
    df = pd.read_csv(f"movies-data/ml-32m/{i}.csv")
    df_list.append(df)


In [ ]:
#Links
df_list[0].head()

##La siguiente información muestra como funciona la info que sale en el dataset de links
"""row = df_list[df_list["movieId"] == 1].iloc[0]

imdb_url = f"https://www.imdb.com/title/tt{int(row['imdbId']):07d}"
tmdb_url = f"https://www.themoviedb.org/movie/{int(row['tmdbId'])}"

print("IMDb:", imdb_url)
print("TMDb:", tmdb_url"""

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [11]:
#Movies 
movies_df = df_list[1]
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#Raiting
df_list[2].head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [ ]:
#Tags
df_list[3].head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


## Información referente a comentarios de sentimientos, estos deben asociarse a peliculas 

In [24]:
enlace = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
response = requests.get(enlace)
response.raise_for_status()


with tarfile.open(fileobj=io.BytesIO(response.content), mode="r:gz") as tar:
    tar.extractall(path="feeling")

In [ ]:
def cargar_reseñas(directorio):
    textos = []
    etiquetas = []

    for label in ['pos', 'neg']:
        path = os.path.join(directorio, label)
        for archivo in os.listdir(path):
            with open(os.path.join(path, archivo), encoding='utf-8') as f:
                textos.append(f.read())
                etiquetas.append(1 if label == 'pos' else 0)
    return textos, etiquetas

# Cargar datos de entrenamiento y prueba
train_texts, train_labels = cargar_reseñas("feeling/aclImdb/train") ##texto de reseñas // ETIQUETA para positivo y negativo
test_texts, test_labels = cargar_reseñas("feeling/aclImdb/test")


## Otro sitio de peliculas

Este servira mas para mostrar datos o complementar los de una pelicula que ya tengamos en el dataset principal

In [45]:
api_key = "fa1b7162" ##Esto es lo importante, no lo borren ni cambien

titulo = "Toy Story" ##Aqui se pone cualquier titulo de pelicla y se ejecuta el codigo

url = f"http://www.omdbapi.com/?t={titulo}&apikey={api_key}&plot=full"

response = requests.get(url)
data = response.json()

if data["Response"] == "True":
    print("Título:", data["Title"])
    print("Año:", data["Year"])
    print("Género:", data["Genre"])
    print("Director:", data["Director"])
    print("Actores:", data["Actors"])
    print("Puntuación IMDb:", data["imdbRating"])
    print("Sinopsis:", data["Plot"])
else:
    print("Película no encontrada")


Título: Toy Story
Año: 1995
Género: Animation, Adventure, Comedy
Director: John Lasseter
Actores: Tom Hanks, Tim Allen, Don Rickles
Puntuación IMDb: 8.3
Sinopsis: A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that his life (as a toy) is good. However, he must worry about Andy's family moving, and what Woody does not know is about Andy's birthday party. Woody does not realize that Andy's mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy's new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips.


In [ ]:
##Esta es para varias peliculas
peliculas = ["Inception", "The Matrix", "Toy Story"]

datos_peliculas = []

for titulo in peliculas:
    url = f"http://www.omdbapi.com/?t={titulo}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if data["Response"] == "True":
        datos_peliculas.append(data)

print(datos_peliculas)

[{'Title': 'Inception', 'Year': '2010', 'Rated': 'PG-13', 'Released': '16 Jul 2010', 'Runtime': '148 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Christopher Nolan', 'Writer': 'Christopher Nolan', 'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page', 'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.', 'Language': 'English, Japanese, French', 'Country': 'United States, United Kingdom', 'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '87%'}, {'Source': 'Metacritic', 'Value': '74/100'}], 'Metascore': '74', 'imdbRating': '8.8', 'imdbVotes': '2,718,935', 'imdbID

Combinación de ambas

In [39]:
df_list[1].size

262755

In [44]:
import re
movies = df_list[1].head(50)
# Función para limpiar título
def limpiar_titulo(titulo):
    # Esto elimina " (1995)" u otros años entre paréntesis
    return re.sub(r"\s*\(\d{4}\)$", "", titulo)

# Aplicar limpieza
movies["clean_title"] = movies["title"].apply(limpiar_titulo)

# Extraer año si quieres
def extraer_ano(titulo):
    match = re.search(r"\((\d{4})\)$", titulo)
    if match:
        return int(match.group(1))
    return None

movies["year"] = movies["title"].apply(extraer_ano)

movies.head()

/tmp/ipykernel_3378/3273266103.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["clean_title"] = movies["title"].apply(limpiar_titulo)
/tmp/ipykernel_3378/3273266103.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["year"] = movies["title"].apply(extraer_ano)


,movieId,title,genres,clean_title,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [ ]:


omdb_url = "http://www.omdbapi.com/"

# 3️⃣ Función para obtener info de OMDb
def obtener_info_omdb(titulo):
    params = {
        "t": titulo,
        "apikey": api_key,
        "plot": "short",  # puedes poner "full" si quieres la sinopsis completa
        "r": "json"
    }
    response = requests.get(omdb_url, params=params)
    data = response.json()
    if data.get("Response") == "True":
        return {
            "Title": data.get("Title"),
            "Year": data.get("Year"),
            "Genre": data.get("Genre"),
            "Director": data.get("Director"),
            "Actors": data.get("Actors"),
            "imdbRating": data.get("imdbRating"),
            "Plot": data.get("Plot")
        }
    else:
        return None

# 4️⃣ Consultar OMDb para todas las películas y guardar resultados
info_peliculas = []
for index, row in movies.iterrows():
    titulo = row["clean_title"]
    info = obtener_info_omdb(titulo)
    if info:
        info["movieId"] = row["movieId"]
        info_peliculas.append(info)
    time.sleep(0.2)  # para no saturar la API (limite de peticiones)

# 5️⃣ Crear DataFrame final
df_omdb = pd.DataFrame(info_peliculas)
##df_omdb.to_csv("movies_omdb.csv", index=False) Esto guarda un csv asi que hay que ver una ruta en caso de que vayamos a usarlo

df_omdb.head()

,Title,Year,Genre,Director,Actors,imdbRating,Plot,movieId
0,Toy Story,1995,"Animation, Adventure, Comedy",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles",8.3,A cowboy doll is profoundly jealous when a new...,1
1,Jumanji,1995,"Adventure, Comedy, Family",Joe Johnston,"Robin Williams, Kirsten Dunst, Bonnie Hunt",7.1,A magic board game summons jungle perils and a...,2
2,Grumpier Old Men,1995,"Comedy, Romance",Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret",6.7,Just as John and Max resolve to save their bel...,3
3,Waiting to Exhale,1995,"Comedy, Drama, Romance",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devine",6.0,"Based on Terry McMillan's novel, this film fol...",4
4,Father of the Bride Part II,1995,"Comedy, Family, Romance",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short",6.1,George Banks must deal not only with his daugh...,5


In [8]:
import requests

url = "https://snap.stanford.edu/data/movies.txt.gz"

# Descargar por streaming y guardar en disco
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open("movies.txt.gz", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Descarga completada")



Descarga completada


In [9]:
import pandas as pd
import gzip

file_path = "movies.txt.gz"
data = []
current_review = {}
max_reviews = 10000  # cambiar según lo que quieras cargar

with gzip.open(file_path, "rt", encoding="latin-1") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith("product/productId:"):
            if current_review:
                data.append(current_review)
                if len(data) >= max_reviews:
                    break  # paramos al llegar al límite
                current_review = {}
            current_review["productId"] = line[len("product/productId: "):]
        elif line.startswith("review/userId:"):
            current_review["userId"] = line[len("review/userId: "):]
        elif line.startswith("review/profileName:"):
            current_review["profileName"] = line[len("review/profileName: "):]
        elif line.startswith("review/helpfulness:"):
            current_review["helpfulness"] = line[len("review/helpfulness: "):]
        elif line.startswith("review/score:"):
            try:
                current_review["score"] = float(line[len("review/score: "):])
            except:
                current_review["score"] = None
        elif line.startswith("review/time:"):
            try:
                current_review["time"] = int(line[len("review/time: "):])
            except:
                current_review["time"] = None
        elif line.startswith("review/summary:"):
            current_review["summary"] = line[len("review/summary: "):] if len(line) > len("review/summary: ") else ""
        elif line.startswith("review/text:"):
            current_review["text"] = line[len("review/text: "):] if len(line) > len("review/text: ") else ""

    if current_review and len(data) < max_reviews:
        data.append(current_review)

df = pd.DataFrame(data)
print(df.head())
print(f"Se cargaron {len(df)} reseñas")



    productId          userId                                 profileName  \
0  B003AI2VGA  A141HP4LYPWMSR            Brian E. Erland "Rainbow Sphinx"   
1  B003AI2VGA  A328S9RN3U5M68                                  Grady Harp   
2  B003AI2VGA  A1I7QGUDP043DG                   Chrissy K. McVay "Writer"   
3  B003AI2VGA  A1M5405JH9THP9                                golgotha.gov   
4  B003AI2VGA   ATXL536YX71TR  KerrLines "&#34;Movies,Music,Theatre&#34;"   

  helpfulness  score        time  \
0         7/7    3.0  1182729600   
1         4/4    3.0  1181952000   
2        8/10    5.0  1164844800   
3         1/1    3.0  1197158400   
4         1/1    3.0  1188345600   

                                             summary  \
0  "There Is So Much Darkness Now ~ Come For The ...   
1  Worthwhile and Important Story Hampered by Poo...   
2                      This movie needed to be made.   
3                  distantly based on a real tragedy   
4  "What's going on down in Juarez and s

In [28]:
df

,productId,userId,profileName,helpfulness,score,time,summary,text
0,B003AI2VGA,A141HP4LYPWMSR,"Brian E. Erland ""Rainbow Sphinx""",7/7,3.0,1182729600,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic..."
1,B003AI2VGA,A328S9RN3U5M68,Grady Harp,4/4,3.0,1181952000,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...
2,B003AI2VGA,A1I7QGUDP043DG,"Chrissy K. McVay ""Writer""",8/10,5.0,1164844800,This movie needed to be made.,The scenes in this film can be very disquietin...
3,B003AI2VGA,A1M5405JH9THP9,golgotha.gov,1/1,3.0,1197158400,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...
4,B003AI2VGA,ATXL536YX71TR,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",1/1,3.0,1188345600,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess..."
...,...,...,...,...,...,...,...,...
9995,B000ANVPPQ,A2WQBT7RS1HSSU,"Wade Jackman ""wpjackman""",1/2,2.0,1075852800,McLintock,"Excellent movie, but the quality of the DVD is..."
9996,B000ANVPPQ,A8MZ1GNMTKWNC,Bradley Olson,1/2,3.0,1060732800,"This is definitely a great movie, lousy DVD",Delta/LaserLight has also put out McLintock on...
9997,B000ANVPPQ,A13H2JJ3GEMJN1,"AntiochAndy ""antiochandy""",1/2,4.0,982454400,"A Good, Funny Movie","McLintock is a romantic comedy first, a wester..."
9998,B000ANVPPQ,A35K9TMGMABTIF,"""servmgr1""",1/2,4.0,930960000,A fun movie.,Classic taming of the shrew out west. Reteams ...


In [30]:
df_amazon_subset = df.head(100).copy()
df_amazon_subset

,productId,userId,profileName,helpfulness,score,time,summary,text
0,B003AI2VGA,A141HP4LYPWMSR,"Brian E. Erland ""Rainbow Sphinx""",7/7,3.0,1182729600,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic..."
1,B003AI2VGA,A328S9RN3U5M68,Grady Harp,4/4,3.0,1181952000,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...
2,B003AI2VGA,A1I7QGUDP043DG,"Chrissy K. McVay ""Writer""",8/10,5.0,1164844800,This movie needed to be made.,The scenes in this film can be very disquietin...
3,B003AI2VGA,A1M5405JH9THP9,golgotha.gov,1/1,3.0,1197158400,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...
4,B003AI2VGA,ATXL536YX71TR,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",1/1,3.0,1188345600,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess..."
...,...,...,...,...,...,...,...,...
95,B000063W1R,A3NWP0SFAC0ZD2,K. Wells,1/1,5.0,1196121600,Superb Acting and Action!,A very well-done film and an excellent cast. ...
96,B000063W1R,A1AFHYBINSG7PH,K. Atkins,1/1,4.0,1193443200,The Count of Monte Cristo,This is an excellent movie and I never read th...
97,B000063W1R,A3N2MVBI1A2I9Y,Jerry,1/1,5.0,1192924800,wow,It is one of the best films I have seen and se...
98,B000063W1R,A3VP6XR88INLRB,M. J. McManus,1/1,4.0,1189641600,"Great ""sleeper"" movie",This is a great movie. That many people don't...


In [34]:
import requests
from bs4 import BeautifulSoup

asin = "B00006HAXW"
url = f"https://www.amazon.com/dp/{asin}"

headers = {"User-Agent": "Mozilla/5.0"}  # necesario para no ser bloqueado
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
title = soup.find(id="productTitle").get_text(strip=True)
print(title)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [36]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)

if "captcha" in response.text.lower():
    print("Amazon bloqueó el acceso, no se puede obtener el título")



Amazon bloqueó el acceso, no se puede obtener el título


In [37]:
soup = BeautifulSoup(response.content, "html.parser")
title_tag = soup.find(id="productTitle")
if title_tag:
    title = title_tag.get_text(strip=True)
else:
    title = None

In [ ]:
! pip install kaggle


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Traceback (most recent call last):
  File "/home/vscode/.local/bin/kaggle", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open ./data/imdb-dataset-of-50k-movie-revi

In [10]:
!chmod 600 ~/.kaggle/kaggle.json

In [11]:


# Asegúrate de que el archivo kaggle.json está en la ruta correcta
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.path.expanduser('~/.kaggle')

# Ejemplo: listar datasets
! kaggle datasets list

ref                                                              title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
yashdevladdha/uber-ride-analytics-dashboard                      Uber Data Analytics Dashboard                        17324552  2025-08-08 11:13:42.920000          45393       1008  1.0              
mdsultanulislamovi/student-stress-monitoring-datasets            Student Stress Monitoring Datasets                      24336  2025-07-24 16:30:01.617000          25129        425  1.0              
msnbehdani/mock-dataset-of-second-hand-car-sales                 Car Sales Dataset: Model, Features, and Pricing        501188  2025-08-20 17:47:58.207000           2434         45  1.0              


In [12]:
! kaggle datasets download -d luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish -p ./data/kaggle.json


Dataset URL: https://www.kaggle.com/datasets/luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish
License(s): other
  0%|                                               | 0.00/52.6M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 52.6M/52.6M [00:00<00:00, 1.55GB/s]


In [3]:
!ls ./data/kaggle.json/

imdb-dataset-of-50k-movie-reviews-spanish.zip


In [4]:
!unzip ./data/kaggle.json/imdb-dataset-of-50k-movie-reviews-spanish.zip -d ./data/imdb-dataset/

Archive:  ./data/kaggle.json/imdb-dataset-of-50k-movie-reviews-spanish.zip
  inflating: ./data/imdb-dataset/IMDB Dataset SPANISH.csv  


In [8]:
csv_path = './data/imdb-dataset/IMDB Dataset SPANISH.csv'

df_imdb = pd.read_csv(csv_path)
df_imdb.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [12]:
import pandas as pd
import numpy as np

# Supongamos que ya tienes tus datasets cargados
# reviews_df: tu dataset de reseñas
# movies_df: tu dataset de películas con columna 'movieId'

# Número de reseñas
n_reviews = len(df_imdb)

# Lista de IDs de películas disponibles
movie_ids = movies_df['movieId'].tolist()

# Asignar aleatoriamente un movieId a cada reseña
df_imdb['movieId'] = np.random.choice(movie_ids, size=n_reviews)

# Revisar
print(df_imdb.head())

   Unnamed: 0                                          review_en  \
0           0  One of the other reviewers has mentioned that ...   
1           1  A wonderful little production. The filming tec...   
2           2  I thought this was a wonderful way to spend ti...   
3           3  Basically there's a family where a little boy ...   
4           4  Petter Mattei's "Love in the Time of Money" is...   

                                           review_es sentiment sentimiento  \
0  Uno de los otros críticos ha mencionado que de...  positive    positivo   
1  Una pequeña pequeña producción.La técnica de f...  positive    positivo   
2  Pensé que esta era una manera maravillosa de p...  positive    positivo   
3  Básicamente, hay una familia donde un niño peq...  negative    negativo   
4  El "amor en el tiempo" de Petter Mattei es una...  positive    positivo   

   movieId  
0   253042  
1    99110  
2   144810  
3   180485  
4   166838  
